Removing all columns except:

'InitialApprovalAmount','ForgivenessAmount', 'CurrentApprovalAmount','review_count', 'lat','lon','JobsReported', 'BorrowerZip','rating'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Loading data
df_business = pd.read_csv('../Machine_Learning_CSVs/encoded_loans_dfV2.0.csv', index_col='LoanNumber')
df_business

,BorrowerZip,LoanStatus,Term,InitialApprovalAmount,CurrentApprovalAmount,JobsReported,ForgivenessAmount,rating,review_count,lat,...,MonthApproved_1,MonthApproved_2,MonthApproved_3,MonthApproved_4,MonthApproved_5,MonthApproved_6,MonthApproved_7,MonthApproved_8,YearApproved_2020,YearApproved_2021
LoanNumber,,,,,,,,,,,,,,,,,,,,,
8728957203,80202,Paid in Full,24,7275742.0,7275742.0,500.0,7356987.79,1.5,7,39.754760,...,0,0,0,1,0,0,0,0,1,0
9826177105,80204,Paid in Full,24,5865500.0,5865500.0,391.0,5943869.60,2.5,3,39.737240,...,0,0,0,1,0,0,0,0,1,0
2089597208,80202,Paid in Full,24,5114560.0,5114560.0,227.0,5173803.65,3.0,2,39.751680,...,0,0,0,1,0,0,0,0,1,0
8574487102,80230,Paid in Full,24,3527000.0,3527000.0,249.0,3573732.75,1.0,4,39.717893,...,0,0,0,1,0,0,0,0,1,0
7218167001,80216,Exemption 4,60,3230500.0,3230500.0,500.0,971592.57,1.0,1,39.771974,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5431627705,80222,Paid in Full,24,1395.0,1395.0,2.0,1411.43,5.0,8,39.684343,...,0,0,0,0,1,0,0,0,1,0
2773518503,80237,Paid in Full,60,1000.0,1000.0,1.0,1009.84,5.0,5,39.598190,...,0,1,0,0,0,0,0,0,0,1
4250667310,80237,Paid in Full,24,1000.0,1000.0,1.0,1008.25,5.0,5,39.598190,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
filtered_biz_df = df_business.filter(['LoanStatus','InitialApprovalAmount','ForgivenessAmount','CurrentApprovalAmount',
                                      'review_count','lat','lon','JobsReported','BorrowerZip','rating'], axis=1)
filtered_biz_df                 

,LoanStatus,InitialApprovalAmount,ForgivenessAmount,CurrentApprovalAmount,review_count,lat,lon,JobsReported,BorrowerZip,rating
LoanNumber,,,,,,,,,,
8728957203,Paid in Full,7275742.0,7356987.79,7275742.0,7,39.754760,-104.997910,500.0,80202,1.5
9826177105,Paid in Full,5865500.0,5943869.60,5865500.0,3,39.737240,-105.008310,391.0,80204,2.5
2089597208,Paid in Full,5114560.0,5173803.65,5114560.0,2,39.751680,-104.998950,227.0,80202,3.0
8574487102,Paid in Full,3527000.0,3573732.75,3527000.0,4,39.717893,-104.895233,249.0,80230,1.0
7218167001,Exemption 4,3230500.0,971592.57,3230500.0,1,39.771974,-104.995255,500.0,80216,1.0
...,...,...,...,...,...,...,...,...,...,...
5431627705,Paid in Full,1395.0,1411.43,1395.0,8,39.684343,-104.937757,2.0,80222,5.0
2773518503,Paid in Full,1000.0,1009.84,1000.0,5,39.598190,-104.903560,1.0,80237,5.0
4250667310,Paid in Full,1000.0,1008.25,1000.0,5,39.598190,-104.903560,1.0,80237,5.0


In [ ]:
# Create our features
X = filtered_biz_df.drop('LoanStatus', axis=1)

# Create our target
y = filtered_biz_df['LoanStatus']

In [ ]:
X.describe()

,InitialApprovalAmount,ForgivenessAmount,CurrentApprovalAmount,review_count,lat,lon,JobsReported,BorrowerZip,rating
count,1.207000e+03,1.207000e+03,1.207000e+03,1207.000000,1207.000000,1207.000000,1207.000000,1207.000000,1207.000000
mean,1.361594e+05,1.347085e+05,1.359278e+05,69.122618,39.722251,-104.966978,14.321458,80215.137531,4.143331
std,4.136190e+05,4.088977e+05,4.135207e+05,172.585119,0.047072,0.052903,37.148812,19.105867,1.068043
min,5.000000e+02,5.046500e+02,5.000000e+02,1.000000,39.368946,-105.245930,1.000000,80012.000000,1.000000
25%,1.856450e+04,1.810364e+04,1.830850e+04,3.000000,39.692745,-104.994540,2.000000,80205.000000,4.000000
50%,4.290000e+04,4.290235e+04,4.300000e+04,8.000000,39.728643,-104.976669,5.000000,80211.000000,4.500000
75%,1.061915e+05,1.064222e+05,1.061415e+05,44.000000,39.751481,-104.937029,12.000000,80222.000000,5.000000
max,7.275742e+06,7.356988e+06,7.275742e+06,1661.000000,40.036910,-104.670690,500.000000,80401.000000,5.000000


In [ ]:
# Check the balance of our target values
filtered_biz_df['LoanStatus'].value_counts()

Paid in Full    1174
Exemption 4       33
Name: LoanStatus, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)

 # SMOTEEN

In [ ]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  6,   3],
       [ 50, 243]])

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7480091012514221

In [ ]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred))

                    pre       rec       spe        f1       geo       iba       sup

 Exemption 4       0.11      0.67      0.83      0.18      0.74      0.54         9
Paid in Full       0.99      0.83      0.67      0.90      0.74      0.56       293

 avg / total       0.96      0.82      0.67      0.88      0.74      0.56       302



# Naive Random Oversampling

In [ ]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled,y_resampled)

LogisticRegression(random_state=1)

In [ ]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[  6,   3],
       [ 54, 239]])

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.7411831626848692

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                    pre       rec       spe        f1       geo       iba       sup

 Exemption 4       0.10      0.67      0.82      0.17      0.74      0.54         9
Paid in Full       0.99      0.82      0.67      0.89      0.74      0.55       293

 avg / total       0.96      0.81      0.67      0.87      0.74      0.55       302

